In [27]:
import xml.etree.ElementTree as ET
import json
import google.generativeai as genai


GEMINI_API_KEY = "Tu APi key aquí"

# Configure Gemini API
genai.configure(api_key=GEMINI_API_KEY)

def parse_bpmn_to_json(xml_file):
    """
    Converts a BPMN XML file into a simplified JSON format.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()

    ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}

    process_data = {"tasks": [], "gateways": [], "events": [], "flows": []}

    for task in root.findall(".//bpmn:task", ns):
        process_data["tasks"].append({"id": task.get("id"), "name": task.get("name")})

    for gateway in root.findall(".//bpmn:exclusiveGateway", ns):
        process_data["gateways"].append({"id": gateway.get("id"), "type": "Exclusive Gateway"})

    for event in root.findall(".//bpmn:startEvent", ns) + root.findall(".//bpmn:endEvent", ns):
        process_data["events"].append({"id": event.get("id"), "name": event.get("name")})

    for flow in root.findall(".//bpmn:sequenceFlow", ns):
        process_data["flows"].append({
            "id": flow.get("id"),
            "source": flow.get("sourceRef"),
            "target": flow.get("targetRef")
        })

    return json.dumps(process_data, indent=4)


import xml.etree.ElementTree as ET
import json

def bpmn_xml_to_json(file_path):
    namespaces = {
        'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL',
        'bpmndi': 'http://www.omg.org/spec/BPMN/20100524/DI'
    }

    # Parse the BPMN file
    tree = ET.parse(file_path)
    root = tree.getroot()
    elements = []
    flow_node_lanes = {}

    # Process Definitions
    definitions_id = root.get('id')

    # Process Collaboration and Participants
    for collaboration in root.findall('.//bpmn:collaboration', namespaces):
        collab_id = collaboration.get('id')
        elements.append({
            "$type": "bpmn:Collaboration",
            "id": collab_id,
            "$parent": definitions_id
        })

        for participant in collaboration.findall('bpmn:participant', namespaces):
            elements.append({
                "$type": "bpmn:Participant",
                "id": participant.get('id'),
                "name": participant.get('name'),
                "processRef": participant.get('processRef'),
                "$parent": collab_id
            })

        # Process Message Flows
        for message_flow in collaboration.findall('bpmn:messageFlow', namespaces):
            elements.append({
                "$type": "bpmn:MessageFlow",
                "id": message_flow.get('id'),
                "sourceRef": message_flow.get('sourceRef'),
                "targetRef": message_flow.get('targetRef'),
                "$parent": collab_id
            })

    # Process Processes and their elements
    for process in root.findall('.//bpmn:process', namespaces):
        process_id = process.get('id')

        # Process Lanes and flow node mapping
        for lane in process.findall('.//bpmn:lane', namespaces):
            lane_id = lane.get('id')
            flow_node_refs = [ref.text for ref in lane.findall('bpmn:flowNodeRef', namespaces)]

            # Record lane associations for flow nodes
            for ref in flow_node_refs:
                flow_node_lanes.setdefault(ref, []).append(lane_id)

            # Create lane element
            elements.append({
                "$type": "bpmn:Lane",
                "id": lane_id,
                "name": lane.get('name'),
                "flowNodeRef": flow_node_refs
            })

        # Process Flow Nodes (Tasks, Events, Gateways, etc.)
        for node in process.findall('*', namespaces):
            node_type = node.tag.split('}')[-1]  # Extract the type from the tag
            node_id = node.get('id')
            node_name = node.get('name')

            # Common properties for all flow nodes
            node_data = {
                "$type": f"bpmn:{node_type}",
                "id": node_id,
                "name": node_name,
                "lanes": flow_node_lanes.get(node_id, []),
                "$parent": process_id
            }

            # Handle specific properties for different node types
            if node_type == "task":
                node_data["incoming"] = [flow.text for flow in node.findall('bpmn:incoming', namespaces)]
                node_data["outgoing"] = [flow.text for flow in node.findall('bpmn:outgoing', namespaces)]
            elif node_type == "startEvent":
                node_data["outgoing"] = [flow.text for flow in node.findall('bpmn:outgoing', namespaces)]
            elif node_type == "intermediateCatchEvent":
                node_data["incoming"] = [flow.text for flow in node.findall('bpmn:incoming', namespaces)]
                node_data["outgoing"] = [flow.text for flow in node.findall('bpmn:outgoing', namespaces)]
            elif node_type in ["exclusiveGateway", "parallelGateway", "eventBasedGateway"]:
                node_data["incoming"] = [flow.text for flow in node.findall('bpmn:incoming', namespaces)]
                node_data["outgoing"] = [flow.text for flow in node.findall('bpmn:outgoing', namespaces)]

            elements.append(node_data)

        # Process Sequence Flows
        for flow in process.findall('bpmn:sequenceFlow', namespaces):
            elements.append({
                "$type": "bpmn:SequenceFlow",
                "id": flow.get('id'),
                "sourceRef": flow.get('sourceRef'),
                "targetRef": flow.get('targetRef'),
                "$parent": process_id
            })

    return json.dumps(elements, indent=2)




In [28]:
def ask_gemini_about_bpmn(json_model, question):

    prompt = f"""
    You are an expert in business process modeling and the BPMN 2.0 standard. I will give
    you a textual representation of a full BPMN model or only selected elements of a BPMN (e.g., a set of
    tasks and flows) and ask you questions about the process. You are supposed to answer the questions
    based on your understanding of the provided model.
    - Please take the role of a process expert who is familiar with the domain of the provided process,
    and use your domain knowledge to better understand and analyze the process, filling in any missing gaps.


    Here is a BPMN model in JSON format:

    {json_model}

    Please answer in natural language, so that any user not familiar with the BPMN standard can
    understand your answer without any technical knowledge; i.e., avoid technical terms like Task, Gate,
    flow, lane, etc.; rather use natural language to describe the behavior of the underlying process.
    {question}
    """

    model = genai.GenerativeModel("gemini-pro")  # Using Gemini Pro model
    response = model.generate_content(prompt)

    return response.text

In [29]:
# Example usage
bpmn_file = "caso_convalidar.bpmn"  # Replace with your BPMN file
json_model = bpmn_xml_to_json(bpmn_file)

print("BPMN Model converted to JSON:")
print(json_model)

# Ask a question about the BPMN model
question = "List all the activities of this process following the order of excecution"
response = ask_gemini_about_bpmn(json_model, question)

print("\nResponse from LLM:")
print(response)

BPMN Model converted to JSON:
[
  {
    "$type": "bpmn:Collaboration",
    "id": "Collaboration_0w0xj4s",
    "$parent": "Definitions_1npih32"
  },
  {
    "$type": "bpmn:Participant",
    "id": "Participant_0rsvuff",
    "name": "Direcci\u00f3n de docencia",
    "processRef": "Process_0f7244c",
    "$parent": "Collaboration_0w0xj4s"
  },
  {
    "$type": "bpmn:Participant",
    "id": "Participant_09i38fr",
    "name": "Coordinador docente unidad acad\u00e9mica",
    "processRef": null,
    "$parent": "Collaboration_0w0xj4s"
  },
  {
    "$type": "bpmn:Participant",
    "id": "Participant_04pbhjg",
    "name": "Direcci\u00f3n registro acad\u00e9mico",
    "processRef": null,
    "$parent": "Collaboration_0w0xj4s"
  },
  {
    "$type": "bpmn:Participant",
    "id": "Participant_05oof4y",
    "name": "Alumno",
    "processRef": null,
    "$parent": "Collaboration_0w0xj4s"
  },
  {
    "$type": "bpmn:MessageFlow",
    "id": "Flow_13iltzw",
    "sourceRef": "Participant_05oof4y",
    "targ

In [30]:
question = "What happens if the academic unit doesn’t accept the form?"
response = ask_gemini_about_bpmn(json_model, question)
response

'If the academic unit does not accept the form, the coordinator will be notified and he/she will notify the student.\nThe student will be given some time to correct the issue and appeal the decision.\nIf the appeal is not accepted, the convalidation request will be closed.'

In [37]:
question = "What is the suggested flow to easily complete the process?"
response = ask_gemini_about_bpmn(json_model, question)
response

"To easily complete the process, the suggested flow is as follows:\n1. A student submits a request for course validation with the education department.\n2. The education department analyzes the request and checks if the student has previously validated the course.\n3. If the student has not previously validated the course, the education department sends a request to the academic unit to check if the course can be validated.\n4. The academic unit checks if the course can be validated and sends the result back to the education department.\n5. The education department reviews the result and makes a decision on whether to approve or reject the request.\n6. If the request is approved, the education department notifies the student and sends the validation request to the academic registry.\n7. The academic registry adds the course to the student's academic record and notifies the student.\n8. If the request is rejected, the education department notifies the student and provides the reason for

In [41]:
question = "I'm new to the process and am confused about the alternatives for a rejection, is there anything to suggest to the sutdent in case of rejection?"
response = ask_gemini_about_bpmn(json_model, question)
response

"In case the student's equivalency request is rejected, the student will be notified of the rejection. The student has the possibility of appealing the rejection within a certain time frame. If the appeal is received within the specified time frame, it will be reviewed. If the appeal is accepted, the student's equivalency request will be re-evaluated. If the appeal is rejected or not received on time, the student will be notified of the final decision."